# 📱MyDay Snowpark Python Demo❄️
☃️This Demo will use Snowpark to query Snowflake.

☃️Train an ML Model.

☃️Deploy ML Function to Snowflake.

# 🔌 Connect to Snowflake
Below is how you connect to Snowflake

In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.functions import col
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

# Connection parameters to Snowflake
connection_parameters = {
  "account": "sfsenorthamerica-snowhealth",
  "user": "USER_NAME",
  "password": "SNOW_PASS",
  "role": "SNOW_ROLE",
  "database":"DB",
  "warehouse": "WH"
}

# Create snowflake session w/ login creds above
session = Session.builder.configs(connection_parameters).create()

print(session)


# 📝 Describe Snowflake Table
Just about anything you can do via the Snowflake UI you can do via Snowpark + more.

Train AI/ML Models

Do python things

etc

In [2]:
session.table("SNOWHEALTH.HK.DIETARYENERGY").columns

['ID',
 'VALUE',
 'UNIT',
 'UUID',
 'APP',
 'DEVICE',
 'STARTTIME',
 'ENDTIME',
 'EXTERNALUUID',
 'FOODIMAGENAME',
 'FOODMEAL',
 'FOODTYPE',
 'FOODTYPEUUID',
 'FOODUSDANUMBER']

# 🧮 Create Snowpark Dataframe
Create a Snowflake Dataframe

Below we are querying Snowflake - 

We are getting UserID, Date, time of day and Calories

This is a table of calories collected by an iPhone diet tracking application and Snowflake MyDay iPhone App

In [3]:
meal_cals = session.table("SNOWHEALTH.HK.DIETARYENERGY")\
        .select( col('ID'), F.to_date(col('STARTTIME')).name("DATE"), F.to_time(col('STARTTIME')).name("TIME"),\
            col('VALUE'))\
    .filter(col('VALUE') > 0)\
    .groupBy( col('ID'), col('DATE'), col('TIME') )\
    .agg([(col("VALUE"), "sum")]).toDF('ID','DATE','TIME','CALS')

meal_cals.show(10)

--------------------------------------------
|"ID"   |"DATE"      |"TIME"    |"CALS"    |
--------------------------------------------
|YBTSU  |2022-06-01  |09:00:00  |350.0     |
|0AA7D  |2021-02-05  |08:00:00  |98.0      |
|0AA7D  |2021-02-06  |08:51:08  |21.0      |
|0AA7D  |2021-02-15  |16:00:00  |754.0     |
|0AA7D  |2021-02-23  |11:00:00  |29.0      |
|2243   |2021-02-16  |08:00:00  |300.0     |
|1122   |2021-04-11  |19:59:40  |1080.0    |
|1FEPQ  |2022-01-03  |09:00:00  |168.1698  |
|1FEPQ  |2022-01-07  |12:00:00  |649.048   |
|2243   |2021-04-11  |08:00:00  |357.0     |
--------------------------------------------



# 📑Pandas Data Frame - Load Scored Data
The data we are importing has Macros (data which users are giving to us) and those inputs are pre-graded so we can train our model to predict future values.

In [4]:
import pandas as pd
grades = pd.read_csv('DietGraded.csv')
grades.head()

,CAL_DIFF,CARBS,FAT,PROTEIN,CHOL,SALT,SUGAR,GRADE
0,415,221,55,172,0.16,1.20,3.31,5
1,556,271,66,208,0.25,1.87,7.62,5
2,419,210,60,237,0.28,1.78,1.92,5
3,555,264,66,171,0.09,1.79,18.75,5
4,723,289,72,233,0.27,0.76,19.70,5


###  Split Scored Diet (Grade) and Macro Data from Data Frame
Create X and Y for Macros and Scores

In [5]:
x = grades.drop('GRADE', axis=1)
y = grades['GRADE']

# 🦿 Train ML Model from Dataset Above

In [6]:
from sklearn.neural_network import MLPClassifier

mlpc = MLPClassifier(hidden_layer_sizes = (10, 5), max_iter=2000)
mlpc.fit(x.values, y.values)

MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=2000)

# Pretend Predict

In [7]:
Test_Data = [-400, 200, 10, 1, 1, 1, 1]
mlpc.predict( [Test_Data] )[0]

3

# ☁️ Deploy Model to Snowflake Data Cloud as a Python UDF
Function is deployed to Snowflake as a Python User Defined Function (UDF)
This UDF can be called via this python notebook or via Snowlake UI thru SQL

Becuase the function runs on Snowflake it can run on any Virtual Warehouse on Snowflake.

Eventually you can share this function - not yet though.


In [11]:
session.use_database("SH_MARIUS")
session.use_schema("GRADES")


@udf(name="GRADES.PREDICT_DIET_NEUTRAL_NEW", 
     return_type=IntegerType(), 
     packages=["scikit-learn"],
     is_permanent=True, replace=True, 
     stage_location="@GRADES.HEALTH_STAGE",
     input_types=[FloatType(), FloatType(), FloatType(),FloatType(), FloatType(), FloatType(), FloatType(), FloatType()])

def PREDICT_DIET_NEUTRAL_NEW(calAte, calBurn, carbs, fat, protein, chol, salt, sugar):
    return mlpc.predict( [[calAte - calBurn, carbs, fat, protein, chol, salt, sugar]] )[0]

print(PREDICT_DIET_NEUTRAL_NEW)



### Run UDF

In [ ]:
session.sql("select SNOWHEALTH.GRADES.PREDICT_DIET_NEUTRAL(1,1,1,1,1,1,1,1)").collect()